In [ ]:
%pip install spotipy

In [3]:
import spotipy




In [4]:

from spotipy.oauth2 import SpotifyOAuth

scope = 'playlist-modify-public'

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope , client_id=client_id, client_secret=client_secret, redirect_uri='http://localhost:8888/'))

user_id = spotify.me()['id']
user_id

'5uml6ohe28g5kghuxvkcjd3cy'

In [5]:
playlist_url = 'https://open.spotify.com/playlist/2hYW67pOzsuG5y97Mnd8Mu?si=d72fb087e9e54bac'
playlist = spotify.playlist(playlist_url)

In [6]:
tracks_query = playlist['tracks']['href']
songs = spotify.user_playlist_tracks(user_id, playlist_id=playlist_url, fields=None, limit=100, offset=0, market=None)
playlist_name, playlist_description = playlist['name'], playlist['description']
original_songs = [f"{song['track']['name']} by {', '.join([artist['name'] for artist in song['track']['artists']])}" for song in songs['items']]
original_songs_ids = [song['track']['id'] for song in songs['items']]

In [7]:
# write a chatgpt prompt
print (f"Playlist name: {playlist_name}")
print (f"Playlist description: {playlist_description}")
print ("Songs:")
print (*original_songs, sep='\n')
print ("\n")
print ("Suggest songs to add to this playlist (only suggest songs that exist):")

Playlist name: Hits different
Playlist description: some just hit different - as of 9-Feb-2023
Songs:
Why Didn't You Stop Me? by Mitski
Washing Machine Heart by Mitski
Heat Waves by Glass Animals
Nothing From Nothing by Billy Preston
Little Words by The Happy Fits
Buddy, Ryo by Kensuke Ushio
I Am the Antichrist to You by Nu Deco Ensemble, Kishi Bashi
Parking Garage by Tommy Bravos
affection by BETWEEN FRIENDS
Sometimes by Nick Lutsko
What's Wrong by half•alive
I Am Steve by Hey Steve
High by Lokki
They Stay Down Deep by Giselle
I Really Want to Stay at Your House by Rosa Walton, Hallie Coggins
Terrible Times by Bantams
Everything Goes Away by The Vernes
Just Can’t Get Enough by Black Eyed Peas
The Night We Met by Lord Huron
Looking for Love by Riaan Vosloo, Benedic Lamdin
Anime Dream Queen by bedfashion
House of Memories by Panic! At The Disco
ロマンス宣言 by Kaneko Ayano
The Fall by half•alive
The Woods by Sarah and the Sundays
All My Favorite Songs by Weezer
Enemy (with JID) - from the ser

In [8]:
# parse chatgpt output
output = """
Sure! Here are some suggestions for songs to add to the "Hits different" playlist:

    Floating Weeds by Mac Demarco
    My Boys by TWENTY ØNE PILØTS
    Past Life by Trevor Daniel, Selena Gomez
    Sober by Olivia Rodrigo
    In Your Eyes by The Weeknd
    Black Hole by Radio Company
    Good 4 U by Olivia Rodrigo
    The One That You Love by LP
    What A Man Gotta Do by Jonas Brothers
    Mind Games by Adeline
    Fake Fine by The Kid LAROI
    Orange by EOB
    Jump by Van Halen
    Lilacs by Phoebe Bridgers
    Here Comes The Sun by The Beatles
    The Bitter Truth by Evanescence
    Sleeping In The Storm by Liza Anne
    The Man by Taylor Swift
    Wreckless Love by Alec Benjamin
    Stargazing by Kygo, Justin Jesso

Hope these suggestions help you add some great songs to your playlist!
"""

# extract song names and artists into a dictionary
songs = {}
for line in output.splitlines():
    if line.startswith('    '):
        song, artist = line[4:].split(' by ')
        songs[song] = artist
        
# print the songs
print (*[f"{song} by {artist}" for song, artist in songs.items()], sep='\n')



Floating Weeds by Mac Demarco
My Boys by TWENTY ØNE PILØTS
Past Life by Trevor Daniel, Selena Gomez
Sober by Olivia Rodrigo
In Your Eyes by The Weeknd
Black Hole by Radio Company
Good 4 U by Olivia Rodrigo
The One That You Love by LP
What A Man Gotta Do by Jonas Brothers
Mind Games by Adeline
Fake Fine by The Kid LAROI
Orange by EOB
Jump by Van Halen
Lilacs by Phoebe Bridgers
Here Comes The Sun by The Beatles
The Bitter Truth by Evanescence
Sleeping In The Storm by Liza Anne
The Man by Taylor Swift
Wreckless Love by Alec Benjamin
Stargazing by Kygo, Justin Jesso


In [9]:
for song, artist in songs.items():
    print(song, artist)

Floating Weeds Mac Demarco
My Boys TWENTY ØNE PILØTS
Past Life Trevor Daniel, Selena Gomez
Sober Olivia Rodrigo
In Your Eyes The Weeknd
Black Hole Radio Company
Good 4 U Olivia Rodrigo
The One That You Love LP
What A Man Gotta Do Jonas Brothers
Mind Games Adeline
Fake Fine The Kid LAROI
Orange EOB
Jump Van Halen
Lilacs Phoebe Bridgers
Here Comes The Sun The Beatles
The Bitter Truth Evanescence
Sleeping In The Storm Liza Anne
The Man Taylor Swift
Wreckless Love Alec Benjamin
Stargazing Kygo, Justin Jesso


In [10]:
# get the spotify ids for the songs
ids = []

for song, artist in songs.items():
    result = spotify.search(q=f"track:{song} artist:{artist}", type='track')
    if result['tracks']['items']:
        ids.append(result['tracks']['items'][0]['id'])
        
print (ids)

['7MN8RYSofZsFROBlEOAzXq', '7szuecWAPwGoV1e5vGu8tl', '4ZtFanR9U6ndgddUvNcjcG', '6iutkSs88iLzzevj9bdvbz', '4wNIkl5XGiAACjFBlDWuSd', '6Fba9RZtC6vTY814JToDtP', '6dGnYIeXmHdcikdzNNDMm2', '4W01mWRcPENGoy8SIxaSbg', '3RauEVgRgj1IuWdJ9fDs70', '0Zbbxnx4SGGHoIow4PpISP']


In [11]:
output_playlist_name = f"{playlist_name} (suggested by ChatGPT)"
# spotify.user_playlists(user_id, limit=50, offset=0)
user_playlists =  spotify.current_user_playlists(limit=50, offset=0)
# check if playlist already exists in user's library
if any([playlist['name'] == output_playlist_name for playlist in user_playlists['items']]):
    print ("Playlist already exists")
    output_playlist = [playlist for playlist in user_playlists['items'] if playlist['name'] == output_playlist_name][0]
    # add songs to playlist if they don't already exist
else:
    output_playlist = spotify.user_playlist_create(user_id, output_playlist_name, public=True, description=f"{playlist_description} powered by ChatGPT")

current_song_ids = [song['track']['id'] for song in spotify.user_playlist_tracks(user_id, playlist_id=output_playlist['id'], fields=None, limit=100, offset=0, market=None)['items']]
print(current_song_ids)

# add orginal songs to playlist
songs_to_add = []
# add suggested songs to playlist if they don't already exist in output playlist
songs_to_add += [song_id for song_id in ids if song_id not in current_song_ids]
# add original songs to playlist if they don't already exist in output playlist
songs_to_add += [song_id for song_id in original_songs_ids if song_id not in current_song_ids]





print (songs_to_add)
# spotify.user_playlist_add_tracks(user_id, playlist_id=output_playlist['id'], tracks=ids)


[]
['7MN8RYSofZsFROBlEOAzXq', '7szuecWAPwGoV1e5vGu8tl', '4ZtFanR9U6ndgddUvNcjcG', '6iutkSs88iLzzevj9bdvbz', '4wNIkl5XGiAACjFBlDWuSd', '6Fba9RZtC6vTY814JToDtP', '6dGnYIeXmHdcikdzNNDMm2', '4W01mWRcPENGoy8SIxaSbg', '3RauEVgRgj1IuWdJ9fDs70', '0Zbbxnx4SGGHoIow4PpISP', '2nifujUVU1XyhTSHI2kQ8g', '3jjsRKEsF42ccXf8kWR3nu', '6CDzDgIUqeDY5g8ujExx2f', '72PoJMDfdaw9gGECgA9kTZ', '04hsupNh2hvlTgXSMkyc4T', '1Z62ioKpMBkjzzTA7SeAAm', '4uy1qAf0joHhekSziXXxvm', '09T2QbxX6BBB8SeuXQCPgV', '4LyIHEDbAWDeKV0JbVZae9', '6oLgJATx3ECZ7Pcghmz38L', '1LUFqhv7F1Jyy95qeoVnnq', '6YYljGh0nSVRQD9Tu5lb0B', '1Tw34hE2MgUYvG4iQwmWE4', '4JNj5vWvoJohMPyfrfo4Rc', '7mykoq6R3BArsSpNDjFQTm', '7y9aZpf5W9RF4novHwCha4', '4oChPWSb0E9mYvdVUl08sd', '3JA9Jsuxr4xgHXEawAdCp4', '3hRV0jL3vUpRrcy398teAU', '4HvLFRFQY5o7QDLDbmVeem', '1Uit6wCgxjlLs5dFoTkerh', '2DgdHcjWmO3qd50RzuBLgZ', '7ph7HRUsxYcKLRFTaYCQYQ', '2VgQLPaAPaJRIXeL0FIW2q', '22vtUov7GUvA0CcLorrmfQ', '6zVhXpiYbJhLJWmLGV9k1r', '1r9xUipOqoNwggBpENDsvJ', '6BxhzevDe4Mzrabj9Pa1eZ', '5xTWnde

In [12]:

if len(songs_to_add) > 0:
    spotify.user_playlist_add_tracks(user_id, playlist_id=output_playlist['id'], tracks=songs_to_add)